# Telco Churn Classification Model

In [278]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

## Part 1: Data Importing and Pre-processing

In [279]:
df = pd.read_csv('https://raw.githubusercontent.com/USD-502-FinalProject/502-Summer2022-FinalProject/main/telecom_customer_churn.csv', sep=',')
df.head()

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability


In [280]:
df_zip = pd.read_csv('https://raw.githubusercontent.com/USD-502-FinalProject/502-Summer2022-FinalProject/main/telecom_zipcode_population.csv', sep=',')
df_zip.head()

,Zip Code,Population
0,90001,54492
1,90002,44586
2,90003,58198
3,90004,67852
4,90005,43019


In [281]:
# Key statistics of studied variables
df.describe()

,Age,Number of Dependents,Zip Code,Latitude,Longitude,Number of Referrals,Tenure in Months,Avg Monthly Long Distance Charges,Avg Monthly GB Download,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,6361.000000,5517.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,46.509726,0.468692,93486.070567,36.197455,-119.756684,1.951867,32.386767,25.420517,26.189958,63.596131,2280.381264,1.962182,6.860713,749.099262,3034.379056
std,16.750352,0.962802,1856.767505,2.468929,2.154425,3.001199,24.542061,14.200374,19.586585,31.204743,2266.220462,7.902614,25.104978,846.660055,2865.204542
min,19.000000,0.000000,90001.000000,32.555828,-124.301372,0.000000,1.000000,1.010000,2.000000,-10.000000,18.800000,0.000000,0.000000,0.000000,21.360000
25%,32.000000,0.000000,92101.000000,33.990646,-121.788090,0.000000,9.000000,13.050000,13.000000,30.400000,400.150000,0.000000,0.000000,70.545000,605.610000
50%,46.000000,0.000000,93518.000000,36.205465,-119.595293,0.000000,29.000000,25.690000,21.000000,70.050000,1394.550000,0.000000,0.000000,401.440000,2108.640000
75%,60.000000,0.000000,95329.000000,38.161321,-117.969795,3.000000,55.000000,37.680000,30.000000,89.750000,3786.600000,0.000000,0.000000,1191.100000,4801.145000
max,80.000000,9.000000,96150.000000,41.962127,-114.192901,11.000000,72.000000,49.990000,85.000000,118.750000,8684.800000,49.790000,150.000000,3564.720000,11979.340000


In [282]:
# Key statistics of studied variables
df_zip.describe()

,Zip Code,Population
count,1671.000000,1671.000000
mean,93678.992220,20276.384201
std,1817.763591,20689.117300
min,90001.000000,11.000000
25%,92269.000000,1789.000000
50%,93664.000000,14239.000000
75%,95408.000000,32942.500000
max,96161.000000,105285.000000


In [283]:
# Drop "Customer ID", "Latitude", and "Longitude" columns
# These columns do not fit the requirements needed for our analysis
df.drop(["Customer ID", "Zip Code", "Latitude","Longitude"], axis=1, inplace = True)

In [284]:
# Identify the data types of key variables
print(df.info(), "\n")
print(df_zip.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             7043 non-null   object 
 1   Age                                7043 non-null   int64  
 2   Married                            7043 non-null   object 
 3   Number of Dependents               7043 non-null   int64  
 4   City                               7043 non-null   object 
 5   Number of Referrals                7043 non-null   int64  
 6   Tenure in Months                   7043 non-null   int64  
 7   Offer                              7043 non-null   object 
 8   Phone Service                      7043 non-null   object 
 9   Avg Monthly Long Distance Charges  6361 non-null   float64
 10  Multiple Lines                     6361 non-null   object 
 11  Internet Service                   7043 non-null   objec

In [285]:
# Count of missing values per column in churn data frame
print(df.isnull().sum(), "\n")

# Count of missing values per column in zipcode data frame
print(df_zip.isnull().sum())

Gender                                  0
Age                                     0
Married                                 0
Number of Dependents                    0
City                                    0
Number of Referrals                     0
Tenure in Months                        0
Offer                                   0
Phone Service                           0
Avg Monthly Long Distance Charges     682
Multiple Lines                        682
Internet Service                        0
Internet Type                        1526
Avg Monthly GB Download              1526
Online Security                      1526
Online Backup                        1526
Device Protection Plan               1526
Premium Tech Support                 1526
Streaming TV                         1526
Streaming Movies                     1526
Streaming Music                      1526
Unlimited Data                       1526
Contract                                0
Paperless Billing                 

In [286]:
# Display column unique count to check for outliers
print("Gender unique count:", df['Gender'].unique())
print("Married unique count:", df['Married'].unique())
print("Multiple Lines unique count:", df['Multiple Lines'].unique())
print("Internet Type unique count:", df['Internet Type'].unique())
print("Number of Referrals Type unique count:", df['Number of Referrals'].unique())

Gender unique count: ['Female' 'Male']
Married unique count: ['Yes' 'No']
Multiple Lines unique count: ['No' 'Yes' nan]
Internet Type unique count: ['Cable' 'Fiber Optic' 'DSL' nan]
Number of Referrals Type unique count: [ 2  0  1  3  8  9 10  5  4  7  6 11]


In [287]:
# Let X = predictor variable and y = target variable
X1 = pd.DataFrame(df[['Monthly Charge', 'Total Charges', 'Total Long Distance Charges']])
y1 = pd.DataFrame(df[['Avg Monthly Long Distance Charges']])

# Add a constant variable to the predictor variables
X1 = sm.add_constant(X1)
model01 = sm.OLS(y1, X1).fit()
df['Avg Monthly Long Distance Charges'].fillna(sm.OLS(y1, X1).fit(), inplace=True)

In [288]:
# Let X = predictor variable and y = target variable
X2 = pd.DataFrame(df[['Monthly Charge', 'Total Revenue']])
y2 = pd.DataFrame(df[['Multiple Lines']])

# Add a constant variable to the predictor variables
X = sm.add_constant(X2)
model02 = sm.OLS(pd.get_dummies(y2), X2).fit()
df['Multiple Lines'].fillna(sm.OLS(pd.get_dummies(y2), X2).fit(), inplace=True)

In [ ]:
#df['Internet Type'].replace(np.nan, df['Internet Type'].mode(), inplace=True)
#df['Avg Monthly GB Download'].replace(np.nan, df['Avg Monthly GB Download'].mode(), inplace=True)
#df['Online Security'].replace(np.nan, df['Online Security'].mode(), inplace=True)
#df['Online Backup'].replace(np.nan, df['Online Backup'].mode(), inplace=True)
#df['Device Protection Plan'].replace(np.nan, df['Device Protection Plan'].mode(), inplace=True)
#df['Premium Tech Support'].replace(np.nan, df['Premium Tech Support'].mode(), inplace=True)
#df['Streaming TV'].replace(np.nan, df['Streaming TV'].mode(), inplace=True)
#df['Streaming Movies'].replace(np.nan, df['Streaming Movies'].mode(), inplace=True)
#df['Streaming Music'].replace(np.nan, df['Streaming Music'].mode(), inplace=True)
#df['Unlimited Data'].replace(np.nan, df['Unlimited Data'].mode(), inplace=True)
#df['Churn Category'].replace(np.nan, df['Churn Category'].mode(), inplace=True)
#df['Churn Reason'].replace(np.nan, df['Churn Reason'].mode(), inplace=True)

In [ ]:
# Confirm that there are no more NaN values in data frame
print(df.isnull().sum(), "\n")